<a href="https://colab.research.google.com/github/martinrosch/TextAnalytics/blob/main/RAG_Gemma_jpynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

T4 GPU

High RAM

In [1]:
# -*- coding: utf-8 -*-
"""
RAG Model with PDF Input and Gemma in Google Colab

This notebook demonstrates how to build a simple Retrieval-Augmented
Generation (RAG) system. It takes a PDF file, processes its content,
and uses a Gemma model to answer questions based on that content.

Instructions:
1. Upload your PDF file to your Colab session's file system.
   You can do this using the file explorer panel on the left side of Colab.
2. Update the `pdf_path` variable in the "Load and Process PDF" section
   to match the name of your uploaded PDF file.
3. Run each cell sequentially.
4. Enter your questions in the "Ask Questions" section.
"""


'\nRAG Model with PDF Input and Gemma in Google Colab\n\nThis notebook demonstrates how to build a simple Retrieval-Augmented\nGeneration (RAG) system. It takes a PDF file, processes its content,\nand uses a Gemma model to answer questions based on that content.\n\nInstructions:\n1. Upload your PDF file to your Colab session\'s file system.\n   You can do this using the file explorer panel on the left side of Colab.\n2. Update the `pdf_path` variable in the "Load and Process PDF" section\n   to match the name of your uploaded PDF file.\n3. Run each cell sequentially.\n4. Enter your questions in the "Ask Questions" section.\n'

Map Google drive - Check if GPU is avtivated

In [6]:
# Map Google Drive
from google.colab import drive
drive.mount('/content/drive')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mounted at /content/drive
Sun May  4 08:30:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------------

Assign constants

In [7]:
# Path constant file
root_path = "/content/drive/MyDrive/ZHAW/MAIN/TA/ml_project"
sust_report_path = "/content/drive/MyDrive/ZHAW/MAIN/TA/ml_project/sustainability_statements"

sust_report_abb = "/content/drive/MyDrive/ZHAW/MAIN/TA/ml_project/sustainability_statements/ABB Sustainability Statement 2024.pdf"
sust_report_komax = "/content/drive/MyDrive/ZHAW/MAIN/TA/ml_project/sustainability_statements/2025-03-11-GB-Komax-sa-esg-report-2024-DE.pdf"

In [8]:
# @title 1. Install Necessary Libraries
# Install required packages. Use 'faiss-gpu' if you have a GPU runtime.
# faiss-cpu
!pip install -q transformers datasets sentence-transformers pypdf torch accelerate faiss-cpu bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
# !pip freeze > /content/requirements_v0.txt

In [10]:
# !pip install -r /content/requirements_v0.txt

In [11]:
# @title 2. Import Libraries
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
from pypdf import PdfReader
import numpy as np
import textwrap # For formatting the output nicely
from huggingface_hub import login
import pandas as pd
from IPython.display import Markdown
import textwrap

In [12]:
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None)

In [13]:
# --- Hugging Face Login ---
# You can either set the HUGGINGFACE_TOKEN environment variable
# or use notebook_login() which will prompt you.
try:
    # Attempt to get token from Colab secrets (recommended)
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        print("Logging in using Colab secret 'HF_TOKEN'")
        login(token=hf_token)
    else:
        print("HF_TOKEN secret not found. Using notebook_login().")
        from huggingface_hub import notebook_login
        notebook_login()
except ImportError:
    print("Not in Colab environment or google.colab unavailable. Using notebook_login().")
    from huggingface_hub import notebook_login
    notebook_login()


Logging in using Colab secret 'HF_TOKEN'


## 3. Configuration and Model Setup

```
Recommended Embedding Models for RAG
'all-mpnet-base-v2': Remains a top recommendation for RAG. Its strong
 performance in semantic similarity tasks makes it well-suited for retrieving relevant documents based on query embeddings. It offers a good balance between
 accuracy and efficiency.

'multi-qa-mpnet-base-dot-v1': Highly relevant for RAG, especially if your
primary focus is question answering. This model is trained specifically for
this purpose and is designed to capture semantic relationships between
questions and relevant passages.

'msmarco-distilbert-base-v4': This model is explicitly trained on the MS MARCO
dataset, a large-scale dataset for passage ranking and retrieval. Its training
on this dataset makes it particularly well-suited for RAG applications.
```

In [10]:
# --- Configuration ---
model_id = "google/gemma-3-4b-it" # You can change this to other Gemma models like gemma-7b-it if resources allow
# model_id = "google/gemma-3-12b-it" # You can change this to other Gemma models like gemma-7b-it if resources allow
# embedding_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model_id = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embedding_model_id = 'all-mpnet-base-v2'
chunk_size = 1000 # Size of text chunks (in characters)
chunk_overlap = 100 # Overlap between chunks
top_k_results = 5 # Number of relevant chunks to retrieve

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Load Embedding Model ---
print(f"Loading embedding model: {embedding_model_id}")
embedding_model = SentenceTransformer(embedding_model_id, device=device)
print("Embedding model loaded.")

# --- Load Gemma Model and Tokenizer ---
# Use BitsAndBytes for quantization to load larger models on limited resources
quantization_config = BitsAndBytesConfig(load_in_4bit=True) if torch.cuda.is_available() else None

print(f"Loading model: {model_id}")

# 1. Get your Hugging Face token (see instructions above)
# Or, pass the token directly to from_pretrained:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
# Add padding token if it doesn't exist (common for Gemma models)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
    # Important: Resize model embeddings if using a newly added token
    # model.resize_token_embeddings(len(tokenizer)) # Do this after loading the model

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    torch_dtype=torch.float32,
    device_map="auto", # Automatically distribute model across available devices (GPU/CPU)
    token=hf_token  # Pass the token here
    # torch_dtype=torch.bfloat16 # Use bfloat16 for faster computation if supported, requires Ampere GPU or newer
)
# Resize embeddings if pad token was added
if tokenizer.pad_token_id == tokenizer.eos_token_id:
     model.resize_token_embeddings(len(tokenizer))

print("Gemma model and tokenizer loaded.")

Using device: cuda
Loading embedding model: all-mpnet-base-v2
Embedding model loaded.
Loading model: google/gemma-3-4b-it


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gemma model and tokenizer loaded.


## 4. Load and Process PDF

In [11]:
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    print(f"Extracting text from: {pdf_path}")
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"Error: PDF file not found at {pdf_path}. Please upload it and update the path.")
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text: # Ensure text was extracted
             text += page_text + "\n" # Add newline between pages
    print(f"Extracted {len(text)} characters.")
    return text

def chunk_text(text, chunk_size, chunk_overlap):
    """Splits text into overlapping chunks."""
    print(f"Chunking text (size={chunk_size}, overlap={chunk_overlap})...")
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - chunk_overlap
        if start >= len(text): # Ensure we don't go past the end on the next iteration due to overlap
            break
        # Add the last bit if it wasn't captured
        if start + chunk_size - chunk_overlap > len(text) and end < len(text):
             chunks.append(text[start:])
             break

    print(f"Created {len(chunks)} chunks.")
    # Simple heuristic to remove potentially very short/empty chunks resulting from overlap logic
    chunks = [chunk for chunk in chunks if len(chunk.strip()) > chunk_overlap // 2]
    print(f"Filtered to {len(chunks)} non-trivial chunks.")
    return chunks



## 5. Create Embeddings and FAISS Index

In [12]:
def create_vector_store(chunks, embedding_model):
    """Creates embeddings for text chunks and builds a FAISS index."""
    if not chunks:
        print("No text chunks to process. Skipping vector store creation.")
        return None, None

    print("Generating embeddings for text chunks...")
    # Generate embeddings in batches if needed, but for moderate PDFs, this is often fine
    embeddings = embedding_model.encode(chunks, show_progress_bar=True, device=device)
    print(f"Generated {len(embeddings)} embeddings with dimension {embeddings.shape[1]}.")

    # Create FAISS index
    dimension = embeddings.shape[1]
    # Using IndexFlatL2 for simple Euclidean distance search
    index = faiss.IndexFlatL2(dimension)
    # Add the embeddings to the index
    index.add(np.array(embeddings).astype('float32')) # FAISS requires float32 numpy arrays
    print(f"FAISS index created with {index.ntotal} vectors.")
    return index, embeddings



## 6. RAG Query Function

In [13]:
def query_rag(question, vector_index, text_chunks, embedding_model, model, tokenizer, top_k=3, print_output=False):
    """
    Performs RAG: retrieves relevant chunks and generates an answer using Gemma.
    """
    if vector_index is None:
        return "Error: Vector index not created. Cannot perform query."

    if print_output:
        print(f"\n--- Querying RAG System ---")
        print(f"Question: {question}")

    # 1. Embed the question
    if print_output:
        print("Embedding question...")
    question_embedding = embedding_model.encode([question], device=device) # 4b device=device)
    question_embedding = np.array(question_embedding).astype('float32')

    # 2. Retrieve relevant chunks from FAISS
    if print_output:
        print(f"Searching for top {top_k} relevant chunks...")
    # D: distances, I: indices of the nearest neighbors
    distances, indices = vector_index.search(question_embedding, top_k)

    # 3. Get the actual text chunks
    retrieved_chunks = [text_chunks[i] for i in indices[0]] # indices[0] because we searched for one query

    if print_output:
        print("Retrieved Chunks:")
        for i, chunk in enumerate(retrieved_chunks):
            print(f"Chunk {i+1} (Index {indices[0][i]}, Distance {distances[0][i]:.4f}):\n{textwrap.shorten(chunk, width=100, placeholder='...')}\n")

    # 4. Combine context and question for the LLM prompt
    context = "\n\n".join(retrieved_chunks)

    # Construct the prompt using the specific chat template for Gemma-IT models
    # Reference: https://huggingface.co/google/gemma-2b-it#chat-template
    messages = [
        {"role": "user", "content": f"Based on the following context, please answer the question.\n\nContext:\n{context}\n\nQuestion: {question}"},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # print("\n--- Prompt for Gemma ---")
    # print(textwrap.fill(prompt, width=100)) # Optional: print the full prompt
    # print("-------------------------\n")

    # 5. Generate the answer using Gemma
    if print_output:
        print("Generating answer with Gemma...")
    # Ensure the model and inputs are on the same device
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(device) # Adjust max_length if needed

    # Generate text
    with torch.no_grad(): # Disable gradient calculation for inference
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,  # Limit the length of the generated answer
            do_sample=True,      # Enable sampling for more diverse answers
            temperature=0.7,     # Control randomness (lower = more deterministic)
            top_k=50,            # Consider top 50 tokens during sampling
            top_p=0.95           # Use nucleus sampling
        )

    # Decode the generated text, skipping special tokens and the prompt
    # We need to decode only the newly generated tokens, not the input prompt
    generated_token_ids = outputs[0][inputs['input_ids'].shape[1]:]
    answer = tokenizer.decode(generated_token_ids, skip_special_tokens=True)

    if print_output:
        print("\n--- Generated Answer ---")
        print(textwrap.fill(answer, width=80))
        print("------------------------\n")

    return answer

## Chatbot
https://ai.google.dev/gemma/docs/gemma_chat

Define formatting helper functions

In [14]:
def display_chat(prompt, text):
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '> ', predicate=lambda _: True)
  formatted_text = "<font size='+1' color='teal'>🤖\n\n" + text + "\n</font>"
  return Markdown(formatted_prompt+formatted_text)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Building the chatbot

```
<start_of_turn>user\n  ... <end_of_turn>\n
<start_of_turn>model\n ... <end_of_turn>\n
````

Create a chat helper to manage the conversation state

In [15]:
class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines for the Gemma family of models
  documented at https://ai.google.dev/gemma/docs/core/prompt-structure
  """

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_MODEL__ + message + self.__END_TURN__)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    response = query_rag(message, vector_index, text_chunks, embedding_model, model, tokenizer, top_k=top_k_results)
    result = response.replace(prompt, "")  # Extract only the new response
    self.add_to_history_as_model(result)

    return result

## Prepare


*   Load pdf and chunk document
*   Index creation



In [26]:
pdf_path = "/content/ABB Sustainability Statement 2024.pdf"
pdf_path = "/content/2025-03-11-GB-Komax-sa-esg-report-2024-DE.pdf"
pdf_path = sust_report_komax

In [27]:
# --- Execute PDF Processing ---
try:
    document_text = extract_text_from_pdf(pdf_path)
    text_chunks = chunk_text(document_text, chunk_size, chunk_overlap)

    # Display first few chunks (optional)
    print("\n--- First 3 Chunks ---")
    for i, chunk in enumerate(text_chunks[:3]):
        print(f"Chunk {i+1}:{textwrap.shorten(chunk, width=150, placeholder='...')}")

except FileNotFoundError as e:
    print(e)
    # Stop execution if PDF is not found
    text_chunks = None # Ensure variable exists but is None

# --- Execute Index Creation ---
if text_chunks:
    vector_index, chunk_embeddings = create_vector_store(text_chunks, embedding_model)
else:
    vector_index = None
    print("Skipping index creation due to previous error.")

Extracting text from: /content/drive/MyDrive/ZHAW/MAIN/TA/ml_project/sustainability_statements/2025-03-11-GB-Komax-sa-esg-report-2024-DE.pdf
Extracted 174975 characters.
Chunking text (size=1000, overlap=100)...
Created 195 chunks.
Filtered to 195 non-trivial chunks.

--- First 3 Chunks ---
Chunk 1:DRIVING AUTOMATION ESG-Bericht 2024 INHALT 64 Komax Gruppe ESG-Bericht 2024 Nachhaltig, sozial und verantwortungsbewusst 65 Rahmen des ESG-Berichts...
Chunk 2:ensethik und Compliance 112 Lieferketten-Risikomanagement 114 Zusätzliche Informationen 117 Erklärung des Verwaltungsrats und OR-Referenztabelle...
Chunk 3:und drei Milliarden Menschen um 1960 auf über acht Milliarden im Jahr 2024. Der technologische Fort - schritt erfordert zudem immer mehr...
Generating embeddings for text chunks...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Generated 195 embeddings with dimension 768.
FAISS index created with 195 vectors.


## 7. Ask Questions!

In [28]:
print_output = True

In [29]:
# --- Example Usage ---
if vector_index is not None:
    # Ask your first question
    question1 = "What company is mentioned in the text?"
    answer1 = query_rag(question1, vector_index, text_chunks, embedding_model, model, tokenizer, top_k=top_k_results, print_output=print_output)
    print(f"\nQuestion: {question1}\nAnswer: {answer1}\n")

else:
    print("Cannot ask questions as the vector index was not created (likely due to PDF loading error).")


--- Querying RAG System ---
Question: What company is mentioned in the text?
Embedding question...
Searching for top 5 relevant chunks...
Retrieved Chunks:
Chunk 1 (Index 191, Distance 1.0584):
ar (www.komaxgroup.com/organization). Unternehmensführung Informationen zur Unternehmensführung...

Chunk 2 (Index 193, Distance 1.1192):
Unternehmensethik und Compliance 103 112 Achtung der Menschenrechte Unternehmensethik und...

Chunk 3 (Index 9, Distance 1.2746):
tor Relations / ESG Telefon +41 41 455 04 55 communication@komaxgroup.com 68 Komax Gruppe ESG-...

Chunk 4 (Index 192, Distance 1.2983):
ntwortlich. Der ESG-Bericht 2024 enthält die vom Schweizerischen Obligationenrecht (OR)...

Chunk 5 (Index 24, Distance 1.3077):
h Interessierte anmelden können (› Seite 137), halb- jährliche Telefonkonferenzen mit CEO und...

Generating answer with Gemma...

--- Generated Answer ---
The company mentioned in the text is Komax Group.
------------------------


Question: What company is mentioned in

# Prompts

**Avoid :: Instead use**<br>
non-binding :: non binding<br>
2050 :: year 2050<br>
such as the Science Based Targets initiative (SBTi) :: like the Science Based Targets initiative<br>

```
Potential Issues:

Bias in Voting Instructions: The prompts explicitly instruct the model to vote "AGAINST" under certain conditions. While this aligns with the intended logic, it could introduce bias in the model's responses, potentially leading to overly negative or critical evaluations.

"Mandatory" and "Non-Binding": These terms might have specific legal or regulatory connotations. While they seem relevant to the context, ensure they are used appropriately and within the scope of the intended application.

"Material Concerns": This phrase is relatively subjective. It could be beneficial to provide more concrete examples or criteria for what constitutes "material concerns" to guide the model's judgment.

"Correct, High-Quality, Domain Expert": These expectations might be too stringent. LLMs are probabilistic and not always perfectly accurate. Emphasizing these attributes might lead to overconfidence or misinterpretation of the model's capabilities.

Mitigation Strategies
Rephrasing for Neutrality: Instead of directly instructing the model to vote "AGAINST", consider using more neutral language, such as "assess whether the non-financial report meets the criteria for approval" or "identify any potential issues that warrant further investigation."

Providing Clearer Definitions: Include definitions or examples for potentially ambiguous terms like "material concerns" to ensure consistent and reliable responses.

Emphasizing Transparency and Limitations: Acknowledge that the model's responses are based on probabilistic reasoning and the provided context. Encourage critical evaluation of the results and provide opportunities for human review and validation.

In general, while the prompts do not contain obvious "forbidden" words, they could be refined to reduce potential bias, enhance clarity, and promote responsible use of the LLM. By carefully considering the language and framing of your prompts, you can improve the reliability and trustworthiness of the generated responses. I hope this helps! Let me know if you have any other questions.

Sources
www.issgovernance.com/file/policy/latest/emea/Europe-Voting-Guidelines.pdf
````

In [30]:
prompt_context = """**Role:** You are a voting guideline expert and have to make votes according to the guidelines.
                    **Goal:** Your task is to approve of Non-Financial Information Statement/Report. Vote FOR or AGAINST.
                    **Policy Recommodation:** Generally, vote for the approval of mandatory non-financial information statement/report, unless the independent assurance services provider has raised material concerns about the information presented.
                    **Audience:** Provide an answer using the provided contexts. Your answers are correct, high-quality, and written by a domain expert.
                    **Constraint:** Keep the answer below 50 words.
                    **Constraints:** If the provided context does not contain the answer, simply state, The provided context does not have the answer.
"""


In [31]:
prompt_audit_1 = """**Task:** vote AGAINST non-financial information statement/report, if the non-financial reporting is not audited.
                    Provide a brief explanation or quote.
                    Final answer with vote FOR or AGAINST.
"""
prompt_audit_1 = """**Task:** Evaluate the non-financial information statement/report based on whether it has undergone an independent audit.
                    If the report lacks an audit, assess the implications for its reliability and determine the appropriate vote.
                    Provide a concise explanation or relevant quote from the report to support your decision.
                    Conclude with a clear vote: FOR or AGAINST.
"""

chat_audit = ChatState(model)
message = prompt_context + prompt_audit_1
display_chat(message, chat_audit.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>**Role:** You are a voting guideline expert and have to make votes according to the guidelines.
                    **Goal:** Your task is to approve of Non-Financial Information Statement/Report. Vote FOR or AGAINST.
                    **Policy Recommodation:** Generally, vote for the approval of mandatory non-financial information statement/report, unless the independent assurance services provider has raised material concerns about the information presented.
                    **Audience:** Provide an answer using the provided contexts. Your answers are correct, high-quality, and written by a domain expert.
                    **Constraint:** Keep the answer below 50 words.
                    **Constraints:** If the provided context does not contain the answer, simply state, The provided context does not have the answer.
**Task:** Evaluate the non-financial information statement/report based on whether it has undergone an independent audit.
                    If the report lacks an audit, assess the implications for its reliability and determine the appropriate vote. 
                    Provide a concise explanation or relevant quote from the report to support your decision. 
                    Conclude with a clear vote: FOR or AGAINST.
</blockquote></font><font size='+1' color='teal'>🤖

> The provided context details audits conducted on suppliers, including risk assessments and the use of EcoVadis (IQ Plus) modules. However, it doesn’t explicitly state that the *entire* non-financial report has undergone independent assurance.
> 
> Vote: AGAINST.
</font>

Answer Gemini
````
The ABB Sustainability Statement 2024 underwent independent limited assurance by KPMG AG.
KPMG raised no material concerns, concluding that "nothing has come to our attention
that causes us to believe that the Sustainability Information is not prepared,
in all material respects, in accordance with the Sustainability Reporting Criteria".

Vote: FOR
````

In [ ]:
prompt_audit_2 = """**Task:** vote AGAINST non-financial information statement/report, if the non-financial reporting reflecting a non binding item on the meeting agenda.
                    Provide a brief explanation or quote.
                    Final answer with vote FOR or AGAINST.
"""


prompt_audit_2 = """**Task:** Assess the non-financial information statement/report in relation to its binding status on the meeting agenda.
                    If the report is presented as a non-binding item, evaluate the implications and determine the appropriate vote.
                    Provide a concise explanation or relevant quote to support your decision.
                    Conclude with a clear vote: FOR or AGAINST.
"""

message = prompt_context + prompt_audit_2
display_chat(message, chat_audit.send_message(message))

In [ ]:
prompt_netzero_1 = """**Task:** vote AGAINST non-financial information statement/report, if the company has not publicly declared a target to achieve Net Zero greenhouse gas emissions by year 2050 or sooner.
                    Provide a brief explanation or quote.
                    Final answer with vote FOR or AGAINST.
"""
prompt_netzero_1 = """**Task:** Evaluate the company's commitment to achieving Net Zero greenhouse gas emissions by year 2050 or sooner.
                      Determine if the company has publicly declared such a target.
                      If not, assess the implications for its environmental sustainability efforts and recommend an appropriate vote.
                      Provide a concise explanation or relevant quote to support your recommendation.
                      Conclude with a clear vote: FOR or AGAINST.
"""


chat_netzero = ChatState(model)
message = prompt_context + prompt_netzero_1
display_chat(message, chat_netzero.send_message(message))

In [ ]:
prompt_netzero_2 = """**Task:** vote AGAINST non-financial information statement/report, if the company’s Net Zero target does not include Scope 1, Scope 2, and all relevant Scope 3 emissions.
                    Provide a brief explanation or quote.
                    Final answer with vote FOR or AGAINST.
"""

prompt_netzero_2 = """**Task:** Evaluate the company's Net Zero target, specifically focusing on its inclusion of Scope 1, Scope 2, and all relevant Scope 3 emissions.
                      If the target does not encompass all these scopes, assess the implications for the company's comprehensive approach to emissions reduction and determine the appropriate vote.
                      Provide a concise explanation or relevant quote from the report to support your decision.
                      Conclude with a clear vote: FOR or AGAINST.
"""

message = prompt_context + prompt_netzero_2
display_chat(message, chat_netzero.send_message(message))

In [ ]:
prompt_netzero_3 = """**Task:** vote AGAINST non-financial information statement/report, if the company is not taking the steps needed to be aligned with a Net Zero by year 2050 trajectory (e.g. SBTI et al).
                    Provide a brief explanation or quote.
                    Final answer with vote FOR or AGAINST.
"""
prompt_netzero_3 = """**Task:** Evaluate the company's actions and plans in relation to achieving a Net Zero trajectory by year 2050, considering established frameworks like the Science Based Targets initiative.
                      Assess whether the company is taking the necessary steps to align with this trajectory.
                      If not, analyze the implications for its long-term sustainability goals and determine the appropriate vote.
                      Provide a concise explanation or relevant quote from the report to support your decision.
                      Conclude with a clear vote: FOR or AGAINST.
"""


message = prompt_context + prompt_netzero_3
display_chat(message, chat_netzero.send_message(message))

In [ ]:
print(chat_netzero.get_history())